## Purpose
This notebook is for testing the pbf conversion tool to implement different speeds on the OSM ways.

In [11]:
## Test 4

Idea: The memory of the machine is used up as we iterate over the ways because new ways are 
added to the root, thus increasing the length of the loop.
I will test out adding new ways to a separate oot (called newroot) and removing from only
the newroot. Thereby keeping the original root the same size and adding only to the new root. 
Then the output will be the newroot. This way I can also see the difference in size of the root (originally, and the old root)
Basically all changes will ocur to newroot.
    
Result: Test showed much slower performcane than the current Maxspeed Setter. I suspect it is due to two changes: 
        1. searching the new root for the way with the same waID being interated over
        2. 

In [8]:
import xml.etree.ElementTree as ET
import pandas as pd
import gc
from timeit import default_timer as timer
import numpy as np
import copy

In [15]:
######## INPUTS

road_path = '/Users/askalila/Documents/TfC Offline Work/numo/otp/Clutter/Final networks/original/extended_sf.osm'

output_file_name = 'extended_sf_test4.osm'

newwayID_seed = 1000  # This is a potential bug if wayID exists in the osm we are routing over.


In [ ]:
####### Extra INPUTS


In [3]:
######### input paths to average speed csv here
###### Mexico:
# avgspeed = '/Users/askalila/Documents/GitHub/32_wri_numo/4_Tool_Development/2_transform/avg_speeds/mexico_output.csv'
# cityunits = 'kph' # string 'mph' or 'kph'

# ###### Cairo:
# ## check points to copy in line 13 of next block [1660,16600,83000,182000, 270000, 360000]
# avgspeed = '/Users/askalila/Documents/GitHub/32_wri_numo/4_Tool_Development/2_transform/avg_speeds/cairo_output.csv'
# cityunits = 'kph' # string 'mph' or 'kph'


###### San Francisco
avgspeed = '/Users/askalila/Documents/GitHub/32_wri_numo/4_Tool_Development/2_transform/avg_speeds/sf_output.csv'
cityunits = 'mph' # string 'mph' or 'kph'


### for mapbox
# colnames=['osm_start_node_id','osm_end_node_id', 'speed_kph_mean'] 
# speedsDFpeak = pd.read_csv(avgspeed, names=colnames, header=0)

## for uber
speedsDFpeak = pd.read_csv(avgspeed)


In [44]:
# Setting up Element Tree to read in the osm file
tree = ET.parse(road_path)
root = tree.getroot()

In [45]:
newroot = copy.deepcopy(root)

In [18]:
def CopyNodes (originalway, nodesequence, realspeed, units, newwayID, verbose = 'no'):
    """
    Purpose: this function creates a new way with the nodes that have speed and adds attribute to the old and new ways to mark them as synthetic
    
    input:
        originalway: original way element object
        nodesequence: list of node IDs to be copied as strings in list, in the correct direction
        realspeed: mean speed as string or 'no' to skip changing maxspeed tag
        units: string of 'mph' or'kph'
        newwayID: string with a new way ID number
        
    no outputs
    """

    newway = ET.Element('way', attrib = dict(originalway.items())) #creates new way with same attributes

    # add nodes as nd children
    
    for i,anode in enumerate(nodesequence):    
        NodeRef = ET.Element('nd', attrib = {'ref':anode})
        newway.append(NodeRef)
        

    # adds attributes and tags to new way
    newway.set('id', newwayID) #replace orig way id with new id
    newway.set('synthetic','yes')
    newway.extend(originalway.findall("./tag")) #copies over all the exisitng tags

    if realspeed != 'no': 
    
        # add the maxspeed tag from real speed or replace current maxspeed with real speed
        if newway.find('tag/[@k="maxspeed"]') is None:
            speedtag = ET.Element('tag', attrib = {'k':'maxspeed','v':realspeed})
            newway.append(speedtag)   # adds a tag element under way
        else:
            newway.find('tag/[@k="maxspeed"]').attrib['v'] = realspeed

    
    global newroot
    newroot.append(newway) # adds new way to the newroot's xml
    if verbose == 'yes': print('adding new way with ID',newwayID)
    
    
    

In [19]:
def CheckNodesForSpeeds (wayID, originalway, units, verbose = 'no'):
    """
    This function will check the Speeds Dataframe (global variable) for speeds between node pairs in the way.
    
    Global variables that need to be present:
        root
        speedsDFpeak
    inputs: 
        wayID: original way element's ID as string
        nodelist: the way's list of node reference ids as strings
        units: 'mph' or 'kph' string
        newwayID_seed: starting number for new way's to iterate over by 1
    
    function Calls another function to do:
        - creates new way with nodes and speed and copies all metadata and non-speed tags
        - removes the copied nodes from the original way
        - adds attribute to newway "synthetic = yes"
        
        - adds attribute to origin way "Checked_for_speed = yes"
        
    no outputs
    """
    global newwayID_seed

    #create nodelist of node id strings and similar list of floats
    nodelist = []
    nodelistfloat = []
    for node in originalway.iter('nd'):
        
        ref = node.attrib['ref']
        nodelist.append(ref)
        nodelistfloat.append(int(ref))

    # get reverse of nodelist
    reverseNodelistfloat = list(reversed(nodelistfloat))
    reverseNodelist = list(reversed(nodelist))

    # get only speeds with origin AND Destination nodes in the original way
    relevantSpeeds = speedsDFpeak.loc[(speedsDFpeak.osm_start_node_id.isin(nodelistfloat)) & (speedsDFpeak.osm_end_node_id.isin(nodelistfloat))]
    try: 
        relevantSpeedsShort = relevantSpeeds[['osm_start_node_id','osm_end_node_id','speed_mph_mean']]
    except:
        relevantSpeedsShort = relevantSpeeds[['osm_start_node_id','osm_end_node_id','speed_kph_mean']]

        
    nodepairdict = dict()
    
    

    # create new ways from the relevant speeds
    
    for nodes in relevantSpeedsShort.iterrows():
        pair = list(nodes[1])
        startnode, endnode, realspeed_float = pair[0],pair[1], pair[2]
        realspeed = str(realspeed_float)

        # check if its the direction of node sequence or reverse
        if nodelistfloat.index(endnode) > nodelistfloat.index(startnode): #correct sequence     
            startIndex, endIndex = nodelistfloat.index(startnode), nodelistfloat.index(endnode)
            CopyNodes(originalway, nodelist[startIndex:endIndex+1],realspeed,units,str(newwayID_seed))
            newwayID_seed += 1
            nodepairdict[startIndex] = endIndex

        elif nodelistfloat.index(endnode) < nodelistfloat.index(startnode): # reverse
            # check if the reverse nodepair belongs to another way (from DF in Uber, from xml tree in mapbox), 
            # if yes then skip this block
            try: # check if the speed DF has a different way ID for the reverse node direction (n/a to mapbox speed)
                samewayID = wayID == str(list(speedsDFpeak.loc[(speedsDFpeak.osm_start_node_id == startnode) & (speedsDFpeak.osm_end_node_id == endnode)]['osm_way_id'])[0])
            except: # for mapbox, check if the root has only way with the start and end node pair
                
#                 This did not work becuase it is very slow. Skipping it will not have adverse effects on the result
#                 samewayID = len(root.findall("./way/nd/[@ref ='"+ str(round(startnode)) +"'].." and "./way/nd/[@ref ='"+ str(round(endnode)) +"']..")) == 1
                samewayID = True
              
#             samewayID = True
            if samewayID:
                startIndex, endIndex = nodelistfloat.index(startnode), nodelistfloat.index(endnode)
                revstartIndex, revendIndex = reverseNodelistfloat.index(startnode), reverseNodelistfloat.index(endnode)
                CopyNodes(originalway, reverseNodelist[revstartIndex:revendIndex+1],realspeed,units,str(newwayID_seed))
                newwayID_seed += 1

        else: raise Exception('start node or end node not in way nodelist')


    ## remove nodes from original way

    startPos = 0
    endPos = len(nodelist)-1

    data_items = nodepairdict.items()
    data_list = list(data_items)
    nodepairdf = pd.DataFrame(data_list,columns=['start','end'])

    # first, delete nodes from segments at the start of the way

    while len(nodepairdf[nodepairdf.start == startPos]) > 0: #loop over this until it all start segments are removed from original way

        start = startPos
        end = nodepairdf[nodepairdf.start == startPos].values[0][1]

        for i in range(start, end): # delete nodes in orig way from startPos to end-1
            nodeID = nodelist[i]
            try: originalway.remove(originalway.find("./nd/[@ref='"+nodeID+"']"))
            except: pass
            if verbose == 'yes': print('removed node index',i)

        nodepairdf = nodepairdf.drop(nodepairdf[nodepairdf.start == startPos].index[0])  # delete this row from the df
        startPos = end


    # second, delete nodes from segments at the end of the way
    while len(nodepairdf[nodepairdf.end == endPos]) > 0:  
        end = endPos
        start = nodepairdf[nodepairdf.end == endPos].values[0][0]

        for i in range(start+1, end+1): # delete nodes in orig way from startPos +1 to endPos
            nodeID = nodelist[i]
            try: originalway.remove(originalway.find("./nd/[@ref='"+nodeID+"']"))
            except: pass
            if verbose == 'yes': print('removed node index',i)

        nodepairdf = nodepairdf.drop(nodepairdf[nodepairdf.end == endPos].index[0]) # delete this row from the df
        endPos = start



    copypairs =[]
    sortedNodepairdf = nodepairdf.sort_values(by=['start']).values
    
    # third, the remaining rows should only be nodes of middle segments. delete the middle nodes

    for i, pair in enumerate(sortedNodepairdf): 

        if pair[1] < pair[0]: continue # ignore reverse pairs
        if len(copypairs) == 0: 
            copypairs.append((startPos,pair[0]))
            lastEnd = pair[1]
        if i == len(sortedNodepairdf)-1: copypairs.append((pair[1],endPos))
        if pair[0] > lastEnd: copypairs.append((lastEnd,pair[0]))
        if i == len(sortedNodepairdf)-1 and pair[1] < endPos: copypairs.append((pair[1],endPos))

        for i in range(pair[0]+1, pair[1]): # delete nodes in orig way from startPos +1 to endPos -1
            nodeID = nodelist[i]
            try: originalway.remove(originalway.find("./nd/[@ref='"+nodeID+"']"))
            except: pass
            if verbose == 'yes': print('removed node index',i)

        lastEnd = pair[1]

    if verbose == 'yes': print('copy pairs:',copypairs)
    # create copynodes with no realspeed for all pairs in copypairs 
    for pair in copypairs:
        nodesequence = []
        for i in range(pair[0],pair[1]+1): nodesequence.append(nodelist[i])
        CopyNodes(originalway, nodesequence,'no',units, str(newwayID_seed))
        newwayID_seed += 1


In [20]:
number_ofways = len(root.findall('./way'))
number_ofways_newroot = len(newroot.findall('./way'))

print('Total number of ways in the original osm:', number_ofways)
print('Total number of ways in the copy osm:', number_ofways_newroot)

checkpoints = []
for j in [0.01,0.05, 0.1, 0.5, 0.75, 0.9, 0.99]:
    checkpoints.append(round(number_ofways*j))

Total number of ways in the original osm: 394579
Total number of ways in the copy osm: 394579


Steps to follow:
- iterate through all ways
- pass to function to do: 
    - in speeds dataframe, query for nodes in the way as start nodes and end nodes. returns rows with relevant speeds in either direction
    - for each row in the speeds dataframe, copy the start node, end node and all nodes in between in the correct order to a new way. copy over all the attributes and tags of the way to the 'synthetic' ways. add an attribute that is is synthetic
    - if speeds returned is for nodes in oppostie direction, only copy the nodes to a new way if the way ID is the same, otherwise skip this node pair. Assuming the other way that this nodepari belongs to will be iterated over, they will be incorprated then.
    - remove the nodes that were copied to new ways from the original way. This preserves the nodes needed in the original way if only a subset of its nodes were cut out with real speeds. new ways with real speed in the opposite direction are not considered for the removal of nodes from the original way

- in osm, remove ways with one or fewer nd's

In [46]:
## Run the functions on all the ways in the osm xml tree

start = timer()

print('starting speed augmentation...')
for i,originalway in enumerate(root.findall("./way")):
    wayID = originalway.attrib['id']
    neworiginalway = newroot.find("./way/[@id='"+wayID+"']")
    CheckNodesForSpeeds(wayID, neworiginalway, cityunits)
    
    
    if i in checkpoints: 
        bookmark = timer()
        progress = int(i/(number_ofways-1)*100)
        print(f'{progress}% completed in {np.round((bookmark - start)/60)} minutes...')

        
    # delete ways with ref nodes 1 or fewer
    if len(neworiginalway.findall("./nd")) < 2: newroot.remove(neworiginalway)
        
end = timer()
print(f'100% completed in {np.round((end- start)/60)} minutes!')

starting speed augmentation...
1% completed in 7.0 minutes...
5% completed in 38.0 minutes...


KeyboardInterrupt: 

In [10]:
# output it to .osm file

tree.write(output_file_name, encoding="utf-8",xml_declaration=True)

In [38]:
%whos

Variable                Type                          Data/Info
---------------------------------------------------------------
CheckNodesForSpeeds     function                      <function CheckNodesForSpeeds at 0x7ff3af56ce50>
CopyNodes               function                      <function CopyNodes at 0x7ff3af56cee0>
ET                      module                        <module 'xml.etree.Elemen<...>ml/etree/ElementTree.py'>
avgspeed                str                           /Users/askalila/Documents<...>/avg_speeds/sf_output.csv
checkpoints             list                          n=7
cityunits               str                           mph
copy                    module                        <module 'copy' from '/Use<...>3/lib/python3.9/copy.py'>
gc                      module                        <module 'gc' (built-in)>
i                       int                           0
ipython_vars            list                          n=6
j                       float        

In [12]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)


[('newroot', 26167760),
 ('root', 25106064),
 ('speedsDFpeak', 15532976),
 ('road_path', 159),
 ('avgspeed', 149),
 ('output_file_name', 81),
 ('ET', 72),
 ('np', 72),
 ('pd', 72),
 ('timer', 72),
 ('cityunits', 52),
 ('tree', 48),
 ('newwayID_seed', 28)]

In [42]:
for i,originalway in enumerate(root.findall("./way")):
    wayID = originalway.attrib['id']
    ET.dump(originalway)
    if i==2:
        break

<way id="4307859" version="8" timestamp="2018-09-24T04:17:29Z" uid="8254810" user="gireeshn" changeset="62866649">
    <nd ref="519533963" />
    <nd ref="2465053159" />
    <nd ref="5621056940" />
    <nd ref="26066107" />
    <nd ref="26066111" />
    <nd ref="767478544" />
    <nd ref="26066113" />
    <tag k="highway" v="service" />
  </way>
  
<way id="4311275" version="46" timestamp="2021-04-14T09:34:26Z" uid="11335781" user="cpligovka_lyft" changeset="102918742">
    <nd ref="32927063" />
    <nd ref="2262036212" />
    <nd ref="30033688" />
    <nd ref="989840971" />
    <nd ref="989840976" />
    <nd ref="2262036223" />
    <nd ref="29891969" />
    <nd ref="2262493188" />
    <tag k="NHS" v="STRAHNET" />
    <tag k="bicycle" v="no" />
    <tag k="hgv" v="designated" />
    <tag k="hgv:national_network" v="yes" />
    <tag k="highway" v="motorway" />
    <tag k="lanes" v="4" />
    <tag k="maxspeed" v="65 mph" />
    <tag k="name" v="Bayshore Freeway" />
    <tag k="oneway" v=

In [43]:
neworiginalway = newroot.find("./way/[@id='"+wayID+"']")
# neworiginalway = newroot.find("./way/[@id='4307859']")
ET.dump(neworiginalway)

<way id="4311277" version="74" timestamp="2021-11-05T20:45:50Z" uid="33757" user="Minh Nguyen" changeset="113426629">
    <nd ref="26028708" />
    <nd ref="2371003215" />
    <nd ref="26678537" />
    <tag k="bicycle" v="yes" />
    <tag k="cycleway:right" v="lane" />
    <tag k="expressway" v="yes" />
    <tag k="highway" v="primary" />
    <tag k="lanes" v="2" />
    <tag k="maxspeed" v="35 mph" />
    <tag k="name" v="Central Expressway" />
    <tag k="note" v="adding odbl tag to match split of way and remove tainted tags" />
    <tag k="oneway" v="yes" />
    <tag k="ref" v="G6" />
    <tag k="sidewalk" v="no" />
    <tag k="wikidata" v="Q109417354" />
  </way>
  


In [34]:
newroot.remove(neworiginalway)

In [47]:
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)


[('newroot', 21735880),
 ('root', 19836832),
 ('speedsDFpeak', 15532976),
 ('neworiginalway', 328),
 ('originalway', 200),
 ('avgspeed', 149),
 ('road_path', 148),
 ('CheckNodesForSpeeds', 136),
 ('CopyNodes', 136),
 ('checkpoints', 120),
 ('ET', 72),
 ('np', 72),
 ('pd', 72),
 ('timer', 72),
 ('output_file_name', 70),
 ('wayID', 56),
 ('cityunits', 52),
 ('tree', 48),
 ('i', 28),
 ('newwayID_seed', 28),
 ('number_ofways', 28),
 ('number_ofways_newroot', 28),
 ('progress', 28),
 ('bookmark', 24),
 ('j', 24),
 ('start', 24)]